In [0]:
catalog_name = 'demos'
schema_name = 'engine_rul_predictor'

# Use the variable in the SQL command
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_name}")

In [0]:
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {schema_name}")

In [0]:
source_table = 'dataknobs_predictive_maintenance_and_asset_management.datasets.nasa_data_train_test'
target_table = 'nasa_data_train_test'

# Read data from the source table using SQL
df = spark.sql(f"SELECT * FROM {source_table}")

# Overwrite the target table if it exists
df.write.mode("overwrite").saveAsTable(target_table)

In [0]:
from pyspark.sql.functions import count

# Group by 'id', count the number of rows in each group, and order by 'id'
df_grouped = df.groupBy('id').agg(count('*').alias('count')).orderBy('id')

# Display the DataFrame to use built-in visualizations
display(df_grouped)

Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import col, sum as spark_sum

null_counts = df.select([spark_sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
sorted_null_counts = null_counts.selectExpr("stack({0}, {1}) as (column, null_count)".format(len(df.columns), ', '.join(["'{0}', {0}".format(c) for c in df.columns]))).orderBy('null_count')
display(sorted_null_counts)

In [0]:
from pyspark.sql.functions import col, count

# Group by all columns and count the number of duplicates
duplicates = df.groupBy(df.columns).agg(count("*").alias("count")).filter(col("count") > 1)

# Display the DataFrame to use built-in visualizations
display(duplicates)

In [0]:
from pyspark.sql.functions import corr

# Select numeric columns for correlation matrix
numeric_columns = [c for c, t in df.dtypes if t in ['int', 'double']]

# Compute the Pearson correlation matrix
correlation_values = []
for i, col1 in enumerate(numeric_columns):
    for col2 in numeric_columns[i:]:
        corr_value = df.stat.corr(col1, col2)
        correlation_values.append((col1, col2, corr_value))

# Convert the correlation values to a DataFrame for better visualization
correlation_df = spark.createDataFrame(
    correlation_values,
    ["column1", "column2", "correlation"]
)

display(correlation_df)

Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import col, collect_list

# Checking Outliers using IQR method
result_outliers = []

for column in df.columns:
        # Calculate Q1 and Q3
        quantiles = df.approxQuantile(column, [0.25, 0.75], 0.01)
        Q1, Q3 = quantiles[0], quantiles[1]
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR

        # Filter outliers
        lower_outliers = df.filter(col(column) < lower).select(collect_list(col("id")).alias("lower_array")).first()["lower_array"]
        upper_outliers = df.filter(col(column) > upper).select(collect_list(col("id")).alias("upper_array")).first()["upper_array"]

        if lower_outliers or upper_outliers:
            result_outliers.append({
                'variable_name': column,
                'lower_fence': lower,
                'upper_fence': upper,
                'Number_of_obs_less_than_lower_fence': len(lower_outliers),
                'Number_of_obs_greater_than_upper_fence': len(upper_outliers),
                'lower_array': lower_outliers[:10],
                'upper_array': upper_outliers[:10]
            })
        else:
            result_outliers.append({
                'variable_name': column,
                'message': 'No outliers present'
            })

if result_outliers:
    display(result_outliers)
else:
    print("No outliers found in any column.")

Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import col, when

# Function to replace outliers with lower and upper fence
def replace_outliers_with_fence(df, column):
    quantiles = df.approxQuantile(column, [0.25, 0.75], 0.01)
    Q1, Q3 = quantiles[0], quantiles[1]
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    df = df.withColumn(column, when(col(column) < lower, lower)
                      .when(col(column) > upper, upper)
                      .otherwise(col(column)))
    return df

# Apply the function to all numeric columns
for column in numeric_columns:
    df = replace_outliers_with_fence(df, column)

display(df)

In [0]:
filtered_df = df.filter(col("id").isin([1, 2, 3, 4])).select("id", "Cycle", "SensorMeasure1")
display(filtered_df)

Databricks visualization. Run in Databricks to view.

In [0]:
filtered_sensor20_df = df.filter(col("id").isin([1, 2, 3, 4])).select("id", "Cycle", "SensorMeasure20")
display(filtered_sensor20_df)

Databricks visualization. Run in Databricks to view.

In [0]:
filtered_id150_df = df.filter(col("id").isin([150]))
display(filtered_id150_df)

Databricks visualization. Run in Databricks to view.

In [0]:
filtered_id500_df = df.filter(col("id").isin([500]))
display(filtered_id500_df)

Databricks visualization. Run in Databricks to view.